## Edwards Curve Digital Signature Algorithm (EdDSA) In Python From Scratch

### reference

- [Edwards Curve Digital Signature Algorithm (EdDSA) Explained](https://www.youtube.com/watch?v=OfnJGLiLkRk&list=PLsS_1RYmYQQEun1MTwmvbXurqHIJrFJ0e&index=14)
- [Edwards Curve Digital Signature Algorithm (EdDSA) In Python From Scratch](https://www.youtube.com/watch?v=8TnRzFt3-K0&list=PLsS_1RYmYQQEun1MTwmvbXurqHIJrFJ0e&index=15)
- [A Gentle Introduction to Edwards-curve Digital Signature Algorithm (EdDSA)](https://sefiks.com/2018/12/24/a-gentle-introduction-to-edwards-curve-digital-signature-algorithm-eddsa/#google_vignette)

In [76]:
import random
import hashlib

In [77]:
def add_points(P, Q, a, d, p):
    # twisted edwards curve: (a*x^2 + y^2) mod p = (1 + d*x^2*y^2) mod p
    
    x1, y1 = P
    x2, y2 = Q
    
    x3 = (((x1*y2 + y1*x2) %p ) * pow(1 + d*x1*x2*y1*y2, -1 ,p)) % p
    y3 = (((y1*y2 - a*x1*x2) % p) * pow(1 - d*x1*x2*y1*y2, -1, p)) % p
    
    assert (a*x3*x3 + y3*y3) % p == (1 + d*x3*x3*y3*y3) % p
    
    
    return x3, y3


def apply_double_and_add_method(Q, k, a, d, p):
    """KQ = K x Q"""
    addition_point = Q
     
    k_binary = bin(k)[2:] #0b1111111001
     
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]
         
        # doubling - always
        addtion_point = add_points(addition_point, addition_point, a, d, p)
         
        if current_bit == "1":
            addtion_point = add_points(addition_point, Q, a, d, p)
     
    # is_on_curve(addtion_point, p)
     
    return addition_point

In [78]:
# Ed25519, Curve25519
p = pow(2,255) - 19
a = -1


d = -121665/121666
d = (-121665 * pow(121666, -1, p)) %p

In [79]:
# base point G
u = 9
# Gy = (u-1)/ (u+1)
Gy = ( (u - 1) * pow(u+1, -1, p) ) % p
Gx = 15112221349535400772501151409588531511454012693041857206046113283949847762202

G = (Gx, Gy)

In [80]:
Gy  

46316835694926478169428394003475163141307993866256225615783033603165251855960

In [81]:
assert (a*Gx*Gx + Gy*Gy) % p == (1 + d*Gx*Gx*Gy*Gy) % p

In [82]:
H = G
for i in range(0, 20):
    H = add_points(H, G, a, d, p)
    print(f"{i+2}G = {H}")

2G = (24727413235106541002554574571675588834622768167397638456726423682521233608206, 15549675580280190176352668710449542251549572066445060580507079593062643049417)
3G = (46896733464454938657123544595386787789046198280132665686241321779790909858396, 8324843778533443976490377120369201138301417226297555316741202210403726505172)
4G = (14582954232372986451776170844943001818709880559417862259286374126315108956272, 32483318716863467900234833297694612235682047836132991208333042722294373421359)
5G = (33467004535436536005251147249499675200073690106659565782908757308821616914995, 43097193783671926753355113395909008640284023746042808659097434958891230611693)
6G = (34643617590234865996699167120328052565261792237873803846102513686264813449789, 2399184961499513294557607325187831088545696902880432827228757905043131825908)
7G = (9199134265559022971505535402808359556995554859516252602543778295037484220679, 22512087849695599276028560866629687720820254811233262850576678203618951717560)
8G = (4670639078046

#### Geterate Private Key - public key

In [83]:
# Geterate Private Key - public key

private_key = random.getrandbits(256)
public_key = apply_double_and_add_method(G, private_key, a, d, p)

In [84]:
print(private_key)
print(public_key)

69385452460760307309630523037344954914557867788346609336010656167978758774665
(15112221349535400772501151409588531511454012693041857206046113283949847762202, 46316835694926478169428394003475163141307993866256225615783033603165251855960)


#### Sign

In [85]:
# sign

def text_to_int(text):
    encoded_text = text.encode("utf-8")
    hex_text  = encoded_text.hex()
    return int(hex_text, 16)


def hashing(message_int):
    return int(hashlib.sha256(str(message_int).encode("utf-8")).hexdigest(), 16)
    

message = text_to_int("Heelo, world!")
r = hashing(hashing(message) + message) % p

R = apply_double_and_add_method(G, r, a, d, p)

h = (R[0] + public_key[0] + message) % p
s = (r + h * private_key)

# (R, s)

#### Verify

In [86]:
# Verify
# message, (R, s), public_key, a, d, p, G

h = (R[0] + public_key[0] + message) % p


P1 = apply_double_and_add_method(G, s, a, d, p)
P2 = add_points(R, apply_double_and_add_method(public_key, h, a, d, p), a, d, p)


assert  P1[0] == P2[0] and P1[1] == P2[1]

AssertionError: 

In [ ]:
P1

(15112221349535400772501151409588531511454012693041857206046113283949847762202,
 46316835694926478169428394003475163141307993866256225615783033603165251855960)

In [ ]:
P2

(24727413235106541002554574571675588834622768167397638456726423682521233608206,
 15549675580280190176352668710449542251549572066445060580507079593062643049417)

In [ ]:
"""
s = (r + h * private_key)
P1 = sxG
P1 = (r + h * private_key) x G
P1 = rxG + h*private_key*G
P1 = R + h * public_key
"""